# Translation  
Verwendete Modelle:   
[Facebook  M2M100 1.2B ](https://huggingface.co/facebook/m2m100_1.2B) & [mBART-50 many to one multilingual machine translation](https://huggingface.co/facebook/mbart-large-50-many-to-one-mmt)
- In den Daten werden 15 unterschiedliche Sprachen verwendet: ['English' 'French' 'Thai' 'Turkish' 'Urdu' 'Russian' 'Bulgarian' 'German' 'Arabic' 'Chinese' 'Hindi' 'Swahili' 'Spanish' 'Vietnamese' 'Greek']
- [mBART-50 many to one multilingual machine translation](https://huggingface.co/facebook/mbart-large-50-many-to-one-mmt) unterstützt alle Sprachen bis auf 'Bulgarian', 'Greek'
- [Facebook  M2M100 1.2B ](https://huggingface.co/facebook/m2m100_1.2B)   wird für die Sprachen 'Bulgarian' 'Greek' verwendet



## Install

In [ ]:
pip install transformers

In [ ]:
pip install sentencepiece

In [ ]:
pip install pandas

In [ ]:
pip install tqdm

In [ ]:
pip install ipywidgets

In [ ]:
pip install torch

In [ ]:
pip install evaluate

## Data

In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import torch

0it [00:00, ?it/s]


#### Local - GPU Hub HSLU

In [5]:
df_train = pd.read_csv('data/nlp_train.csv')
df_valid = pd.read_csv('data/nlp_valid.csv')
df_test = pd.read_csv('data/test.csv')

In [6]:
df_train.shape

(10908, 6)

## Model

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

In [ ]:
# Übersetzung für die Sprachen:
# 'Bulgarian' 'Greek'
model_2m2 = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_1.2B")
tokenizer_2m2 = M2M100Tokenizer.from_pretrained("facebook/m2m100_1.2B")

In [ ]:
# Übersetzung für die Sprachen:
# 'English' 'French' 'Thai' 'Turkish' 'Urdu' 'Russian' 'German' 'Arabic' 'Chinese' 'Hindi' 'Swahili' 'Spanish' 'Vietnamese'
model_mbart = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")
tokenizer_mbart = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")

In [ ]:
model_2m2.eval()  # put the model into evaluation mode
model_2m2.to('cuda')  # move the model to GPU

In [ ]:
model_mbart.eval()  # put the model into evaluation mode
model_mbart.to('cuda')  # move the model to GPU

## Translation

In [ ]:
# Methode Übersetzung String/Satz
def sentence_translation_all(text, src_lang): #text = Zu übersetzender String/Satz; src_lang = Sprach-Code in welcher Sprache der Satz vorliegt
    if src_lang == "en_XX": # English muss nicht übersetzt werden
        translation = text
        
    elif src_lang == "bg" or src_lang == "el": # Bulgarian "bg" und Greek "el" wird durch "facebook/m2m100_1.2B" übersetzt
        tokenizer_2m2.src_lang = src_lang
        encoded_de = tokenizer_2m2(text, return_tensors="pt")
        encoded_de.to('cuda')
        with torch.no_grad():
          generated_tokens = model_2m2.generate(**encoded_de, forced_bos_token_id=tokenizer_2m2.get_lang_id("en"))
        translation = tokenizer_2m2.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    else: # alle anderen Sprachen werden durch "facebook/mbart-large-50-many-to-one-mmt" übersetzt
        tokenizer_mbart.src_lang = src_lang
        encoded_de = tokenizer_mbart(text, return_tensors="pt")
        encoded_de.to('cuda')
        with torch.no_grad():
          generated_tokens = model_mbart.generate(**encoded_de)
        translation = tokenizer_mbart.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        
    return translation

In [ ]:
# Zuweisung Dataframe: Welches Dataset soll Übersetzt werden
df = df_train

In [ ]:
# Dataframe leere Spalten für die Übersetzung hinzufügen
df['premise_EN'] = np.nan
df['hypothesis_EN'] = np.nan
df['src_lang'] = np.nan

In [ ]:
# Model-Spezifischer Sprache-Code anpassen
df["src_lang"] = df["lang_abv"].replace(['en', 'fr', 'th', 'tr', 'ur', 'ru', 'de', 'ar', 'zh', 'hi', 'sw', 'es', 'vi'],
                                        ['en_XX', 'fr_XX', 'th_TH', 'tr_TR', 'ur_PK', 'ru_RU', 'de_DE', 'ar_AR', 'zh_CN', 'hi_IN', 'sw_KE', 'es_XX', 'vi_VN'])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# premise Übersetzen
df['premise_EN'] = df.progress_apply(lambda x: sentence_translation_all(x['premise'], x['src_lang']), axis=1)

In [ ]:
# hypothesis Übersetzen
df['hypothesis_EN'] = df.progress_apply(lambda x: sentence_translation_all(x['hypothesis'], x['src_lang']), axis=1)

In [ ]:
df.head(20)

In [ ]:
df.shape

In [ ]:
#Das gleiche Prinzip wurde für das Validation- und Testset gemacht
df.to_csv('data/EN/nlp_train_en_v2.csv')